In [43]:
import pygame 
import logging
import random

# Constants for the tile size
TILE_SIZE = 16
UI_WIDTH = 200                  # Width of the UI section
UI_HEIGHT = 600                 # Height
TRACTOR_SIZE = 12
WHITE = (255, 255, 255)
LIGHT_BROWN = (210, 180, 140)   # Light brown for un-ploughed soil
DARK_BROWN = (101, 67, 33)      # Dark brown for ploughed soil
GREEN = (0, 255, 0)             # Green for planted crops
RED = (255, 0, 0) 
TILE_SIZE = 16                  # Constants for the tile size

# default field
field_height = 20 # min 20
field_width = 20 # min 1


class Field:
    def __init__(self, height, width):
        self.width = width
        self.height = height
        self.layout = self.generate_field(height, width)

    def generate_field(self, height, width):
        height += 2
        width += 2

        field = [['grass' for _ in range(width)] for _ in range(height)]

        grass_corner_tl = 'soil_bottom_right'
        grass_corner_tr = 'soil_bottom_left'
        grass_corner_bl = 'soil_top_right'
        grass_corner_br = 'soil_top_left'
        grass_edge_top = 'soil_bottom'
        grass_edge_bottom = 'soil_top'
        grass_edge_left = 'soil_right'
        grass_edge_right = 'soil_left'
        soil_variations = ['soil1', 'soil2', 'soil3']

        field[1][1] = grass_corner_tl
        field[1][width - 2] = grass_corner_tr
        field[height - 2][1] = grass_corner_bl
        field[height - 2][width - 2] = grass_corner_br

        for x in range(width - 2):
            field[1][x] = grass_edge_top
            field[height - 2][x] = grass_edge_bottom
        for y in range(height - 2):
            field[y][1] = grass_edge_left
            field[y][width - 2] = grass_edge_right

        for y in range(2, height - 2):
            for x in range(2, width - 2):
                field[y][x] = random.choice(soil_variations)

        return field

    def plough_field(self, row, col):
        # Check if the row and col indices are within the valid range
        if 0 <= row < len(self.layout) and 0 <= col < len(self.layout[0]):
            # Adjust for border offset
            self.layout[row + 2][col + 2] = 'ploughed_soil'
        else:
            # Optionally log an error or handle the invalid index case
            logging.error(f"Invalid plough field indices: row {row}, col {col}")

    def get_truncated_layout(self):
        return [row[2:-2] for row in self.layout[2:-2]]

    def render(self, screen, tileset):
        for y, row in enumerate(self.layout):
            for x, tile_name in enumerate(row):
                if tile_name == 'ploughed_soil':
                    screen.blit(darker_soil_tile, (x * TILE_SIZE, y * TILE_SIZE))
                else:
                    render_tile(screen, tile_name, x, y, tileset)

class Tractor:
    def __init__(self, row, col, tractor_colour):
        self.row = row
        self.col = col
        self.border_offset = 2
        self.tractor_colour = tractor_colour
        # Load the tractor image based on the specified colour
        self.tractor_image = pygame.image.load(f'assets/{tractor_colour}_tractor_12x12.png')

    def draw(self, win):
        # Adjust position to include border offset
        x = (self.col + self.border_offset) * TILE_SIZE + (TILE_SIZE - TRACTOR_SIZE) // 2
        y = (self.row + self.border_offset) * TILE_SIZE + (TILE_SIZE - TRACTOR_SIZE) // 2
        # Draw the tractor image
        win.blit(self.tractor_image, (x, y))

    def move_to(self, position):
        self.row, self.col = position

class Button:
    def __init__(self, x, y, width, height, text):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text

    def draw(self, screen):
        # Draw the game 
        pygame.draw.rect(screen, (59, 130, 246), self.rect)
        font = pygame.font.Font(None, 30)
        text_surf = font.render(self.text, True, (255, 255, 255))
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)
        
    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)
    
    
# Function declarations

def get_tile_by_name(tileset, name, tile_width, tile_height):
    x, y = reversed_tile_names[name]
    rect = pygame.Rect(x * tile_width, y * tile_height, tile_width, tile_height)
    image = pygame.Surface(rect.size, pygame.SRCALPHA).convert_alpha()
    image.blit(tileset, (0,0), rect)
    return image

def render_tile(screen, tile_name, x, y, tileset):
    if tile_name:
        tile_image = get_tile_by_name(tileset, tile_name, TILE_SIZE, TILE_SIZE)
        screen.blit(tile_image, (x * TILE_SIZE, y * TILE_SIZE))
        
def start_tractor(field):
    # Starting position within the ploughable area
    tractor = Tractor(2, 2, tractor_colour)   # No need to offset by border size anymore
    plough_movements = student_function(field)
    all_movements = plough_movements
    return tractor, all_movements

def render_score(screen, score, x, y):
    font = pygame.font.Font(None, 36)
    text = font.render(f"Score: {score}", True, (0, 0, 0))
    screen.blit(text, (x, y))
    
def print_field(field):
    truncated_layout = field.get_truncated_layout()
    for i, row in enumerate(truncated_layout):
        row_str = ', '.join([cell.replace('soil1', 'soil') \
            .replace('soil2', 'soil') \
                .replace('soil3', 'soil') for cell in row])
        if i == 0:
            print(f"[[{row_str}],")
        elif i == len(truncated_layout) - 1:
            print(f" [{row_str}]]")
        else:
            print(f" [{row_str}], ")
            
def print_movement(field):
    print(student_function(field.get_truncated_layout()))
    
def student_function(field):
    plough_movements = []
    for row in range(len(field)):
        plough_movements.append((row, 0))
    return plough_movements



logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

pygame.init()

tile_names = {
    (0,0): "soil_top_left", (2, 0): "soil_top_right",
    #(3, 0): "diag_tl__br", (4, 0): "diag_bl_tr", 
    (5, 0): 'soil1', (6, 0): "soil2", (7, 0): "soil3",
    (0, 1): "soil_left", (1, 1): "grass", (2, 1): "soil_right", 
    (3,1): "soil_bottom_right", (4, 1): "soil_bottom_left", 
    (6, 1): "soil_top_left", (7, 1): "soil_top_right", 
    (8, 1): "soil_top", #(10, 1): "soil_left", 
    (1, 2): "soil_bottom"
}

reversed_tile_names = {v: k for k, v in tile_names.items()}

# Load the darker soil tile
darker_soil_tile = pygame.image.load('assets/soil_tile_darker.png')

field = Field(field_width + 2, field_height + 2)
field_layout = field.get_truncated_layout()
movement_list = student_function(field.get_truncated_layout())

def main():
    global movement_list
    pygame.init()
    clock = pygame.time.Clock()
    
    # Load the tileset image
    tileset = pygame.image.load('assets/free.png')
    eai_image = pygame.image.load('assets/EAI_Blue_Dark.png')
    
    # Scale down EAI image
    scale_factor = 0.3
    scaled_eai_width = int(eai_image.get_width() * scale_factor)
    scaled_eai_height = int(eai_image.get_height() * scale_factor)
    eai_image = pygame.transform.scale(eai_image, (scaled_eai_width, scaled_eai_height))
    
    # Initialize variables
    tractor = None
    current_movement_list = []
    movement_index = 0
    tractor_active = False
    score = 0
    score_shift = 15
    
    # Calculate the display size (including UI)
    rows, cols = len(field.layout) - 4, len(field.layout[0]) - 4
    display_width = TILE_SIZE * (cols + 4) + UI_WIDTH
    display_height = TILE_SIZE * (rows + 4)
    if display_height < 400:
        display_height = 400
    screen = pygame.display.set_mode((display_width, display_height))
    
    ui_center_x = TILE_SIZE * cols + (UI_WIDTH // 2)
    
    # Initialise variables
    button_width, button_height = 100, 50
    start_button = Button(ui_center_x + 13, 100, button_width, button_height, "Sart")
    stop_button = Button(ui_center_x + 13, 200, button_width, button_height, "Stop")
    
    # Position for EAI image
    eai_image_x = ui_center_x + 60 - scaled_eai_width // 2
    eai_image_y = 10
    
    # Main game loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if start_button.is_clicked(event.pos):
                    print("Starting Tractor")
                    tractor, current_movement_list = start_tractor(field.get_truncated_layout())
                    movement_index = 0
                    tractor_active = True
                elif stop_button.is_clicked(event.pos):
                    print("Stop button clicked")
                    running = False
                
        screen.fill(WHITE)  # clear the screen
        # Draw the EAI image at the top
        screen.blit(eai_image, (eai_image_x, eai_image_y))
        # Draw the buttons
        start_button.draw(screen)
        stop_button.draw(screen)
        # Render the field
        field.render(screen, tileset)
        # Render score
        render_score(screen, score, ui_center_x + score_shift, 300)
        
        # Animate tractor movement
        if tractor_active and tractor:
            if movement_index < len(current_movement_list):
                # Move the tractor
                prev_position = tractor.row, tractor.col
                next_position = current_movement_list[movement_index]
                tractor.move_to(next_position)
                
                # Plough the field at the tractor's new position
                field.plough_field(next_position[0], next_position[1])
                
                # Update score
                score += 1
                if next_position[0] != prev_position[0]:
                    if (next_position[0] % 2 == 0 and next_position[1] != 0) or \
                        (next_position[0] % 2 == 1 and next_position[1] != len(field.get_truncated_layout()[0]) - 1):
                            score += 20  # Apply penalty
                            
                movement_index += 1
            else:
                tractor_active = False
                
        if tractor:
            tractor.draw(screen)
            
        pygame.display.flip()
        clock.tick(simulation_speed)
        
    pygame.quit()

In [44]:
simulation_speed = 40
tractor_colour = 'pink'
field_height = 5 # min 1
field_width = 5 # min 1

In [45]:
field_5_by_5 = [
    ['soil', 'soil', 'soil', 'soil', 'soil'],
    ['soil', 'soil', 'soil', 'soil', 'soil'],
    ['soil', 'soil', 'soil', 'soil', 'soil'],
    ['soil', 'soil', 'soil', 'soil', 'soil'],
    ['soil', 'soil', 'soil', 'soil', 'soil'],
]

In [46]:
field_height = 10
field_width = 5

field = Field(field_width + 2, field_height + 2) # We create a new field with this line that is two tiles wider than the values we gave, which makes room for some grass.
print_field(field) # field is 10x5

[[soil, soil, soil, soil, soil, soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil, soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil, soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil, soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil, soil, soil, soil, soil, soil]]


In [47]:
# We can change the field dimensions by altering the field's parameters.
field_height = 5
field_width =  6

field = Field(field_width + 2, field_height + 2)
print_field(field)

[[soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil]]


In [48]:
# Moving through the field
field_height = 5
field_width = 5

field = Field(field_width + 2, field_height + 2)
print_field(field)

def student_function(field):
    plough_movements = []
    for row in range(len(field[0])):
        plough_movements.append((0, row))
        
    return plough_movements

print_movement(field)

[[soil, soil, soil, soil, soil],
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil], 
 [soil, soil, soil, soil, soil]]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]


In [49]:
field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Starting Tractor
Stop button clicked


In [50]:
def student_function(field):
    plough_movements = []
    for col in range(len(field)):
        for row in range(len(field[col])):
            plough_movements.append((col, row))
            
    return plough_movements

In [51]:
# soil1, soil2 and soil3 are used to draw variations of a soil tile in Pygame. They have no meaning to our code.

field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],  
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (9, 0),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4)]

In [52]:
field_wide = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2', 'soil3', 'soil3', 'soil1', 'soil2', 'soil2', 'soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil3', 'soil3', 'soil1', 'soil2','soil2', 'soil3', 'soil3']
]

student_function(field_wide)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6)]

In [54]:
field = Field(field_width + 2, field_height + 2)

if __name__ == "__main__":
    main()

Starting Tractor
Stop button clicked


In [87]:
def reverse_list(input_list):
    # Insert your code here
    reversed_list = input_list[::-1]
    # if (len(input_list) == 1):  # recursing arrays
    #     return input_list
    # else:
    #     return [input_list[-1]] + reverse_list(input_list[:-1])
    return reversed_list

In [72]:
current_row = [0, 1, 2, 3, 4]

reverse_list(current_row)

[4, 3, 2, 1, 0]

In [73]:
current_row = [1, 3, 2, 0, 4]

reverse_list(current_row)

[4, 0, 2, 3, 1]

In [74]:
def student_function(field):
    plough_movements = []
    for col in range(len(field)):
        for row in range(len(field[col])):
            if row % 2 == 0:
                reverse_list(row)
                plough_movements.append((col, row))
            else:
                plough_movements.append((col, row))
                
    return plough_movements

In [ ]:
def student_function(field):
    plough_movements = []
    for idx, col in enumerate(field):
        

In [95]:
def student_function(field):
    plough_movements = []
    for row in range(len(field)):
        if row % 2 == 0:
            # Even row: left to right
            cols = range(len(field[row]))
        else:
            cols = reverse_list(range(len(field[row])))
            
        for col in cols:
            plough_movements.append((col, row))
            
        # after completing ploughing a row, if not the last row
        if row < len(field) - 1:
            # Move straight down to the row column at the same column index
            next_col = cols[-1] if isinstance(cols, list) else list(cols)[-1]
            plough_movements.append((row + 1, next_col))
            
    return plough_movements

In [96]:
field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (1, 4),
 (4, 1),
 (3, 1),
 (2, 1),
 (1, 1),
 (0, 1),
 (2, 0),
 (0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (3, 4),
 (4, 3),
 (3, 3),
 (2, 3),
 (1, 3),
 (0, 3),
 (4, 0),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (4, 4),
 (5, 4),
 (4, 5),
 (3, 5),
 (2, 5),
 (1, 5),
 (0, 5),
 (6, 0),
 (0, 6),
 (1, 6),
 (2, 6),
 (3, 6),
 (4, 6),
 (7, 4),
 (4, 7),
 (3, 7),
 (2, 7),
 (1, 7),
 (0, 7),
 (8, 0),
 (0, 8),
 (1, 8),
 (2, 8),
 (3, 8),
 (4, 8),
 (9, 4),
 (4, 9),
 (3, 9),
 (2, 9),
 (1, 9),
 (0, 9)]

In [90]:
for i, col in enumerate(field_standard):
    # print(col)
    if i % 2 == 0:
        for j, row in enumerate(col):
            plough_movements.append((i, j))
    elif i % 2 != 0:
        for j, row in enumerate(col):
            row = reverse_list(row)
            plough_movements.append((i, j))
            
print(plough_movements)
    # for j, row in enumerate(col):
    #     print(f"{j}:{row}")

[(['soil3', 'soil3', 'soil1', 'soil2', 'soil2'], 'soil3'), (['soil3', 'soil3', 'soil1', 'soil2', 'soil2'], 'soil3'), (['soil3', 'soil3', 'soil1', 'soil2', 'soil2'], 'soil1'), (['soil3', 'soil3', 'soil1', 'soil2', 'soil2'], 'soil2'), (['soil3', 'soil3', 'soil1', 'soil2', 'soil2'], 'soil2'), (['soil2', 'soil3', 'soil2', 'soil1', 'soil3'], '2lios'), (['soil2', 'soil3', 'soil2', 'soil1', 'soil3'], '3lios'), (['soil2', 'soil3', 'soil2', 'soil1', 'soil3'], '2lios'), (['soil2', 'soil3', 'soil2', 'soil1', 'soil3'], '1lios'), (['soil2', 'soil3', 'soil2', 'soil1', 'soil3'], '3lios'), (['soil2', 'soil3', 'soil1', 'soil3', 'soil3'], 'soil2'), (['soil2', 'soil3', 'soil1', 'soil3', 'soil3'], 'soil3'), (['soil2', 'soil3', 'soil1', 'soil3', 'soil3'], 'soil1'), (['soil2', 'soil3', 'soil1', 'soil3', 'soil3'], 'soil3'), (['soil2', 'soil3', 'soil1', 'soil3', 'soil3'], 'soil3'), (['soil3', 'soil2', 'soil3', 'soil3', 'soil2'], '3lios'), (['soil3', 'soil2', 'soil3', 'soil3', 'soil2'], '2lios'), (['soil3', 's

In [70]:
field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

TypeError: 'int' object is not subscriptable

In [76]:
field_standard = [
    ['soil3', 'soil3', 'soil1', 'soil2', 'soil2'],
    ['soil2', 'soil3', 'soil2', 'soil1', 'soil3'],
    ['soil2', 'soil3', 'soil1', 'soil3', 'soil3'],
    ['soil3', 'soil2', 'soil3', 'soil3', 'soil2'],
    ['soil3', 'soil2', 'soil1', 'soil3', 'soil2'],
    ['soil2', 'soil3', 'soil3', 'soil1', 'soil3'],
    ['soil3', 'soil1', 'soil3', 'soil2', 'soil2'],
    ['soil2', 'soil1', 'soil2', 'soil2', 'soil1'],
    ['soil1', 'soil1', 'soil3', 'soil2', 'soil3'],
    ['soil3', 'soil3', 'soil3', 'soil1', 'soil1']
]

student_function(field_standard)

TypeError: object of type 'int' has no len()

In [ ]:
import pandas as pd